In [ ]:
import sys
print(sys.executable)

In [ ]:
!pip install --force-reinstall --no-deps --upgrade git+https://github.com/unimib-datAI/Semtui-python.git

!pip install --upgrade --no-cache-dir git+https://github.com/unimib-datAI/Semtui-python.git

In [ ]:
import os
import SemT_py
import json
import numpy as np
import pandas as pd

# Import necessary classes and functions from the SemT_py package
from SemT_py.token_manager import TokenManager
from SemT_py.extension_manager import ExtensionManager
from SemT_py.reconciliation_manager import ReconciliationManager
from SemT_py.utils import Utility
from SemT_py.dataset_manager import DatasetManager
from SemT_py.modification_manager import ModificationManager

# Set pandas display options

pd.set_option('display.max_columns', None)  # Display all columns
pd.set_option('display.max_rows', 20)  # Limit to 20 rows for display

In [ ]:

# Initialize the parser
config = configparser.ConfigParser()

# Read the credentials file
config.read("credentials.txt")

# Access the values
base_url = config["DEFAULT"]["BASE_URL"]
api_url = config["DEFAULT"]["API_URL"]
username = config["DEFAULT"]["USERNAME"]
password = config["DEFAULT"]["PASSWORD"]


token_manager = TokenManager(api_url, username, password)
# Get the token
token = token_manager.get_token()
utility = Utility(base_url, token_manager)

In [1]:
import csv

# Define sample biodata and reviews with additional entity linking columns
data = [
    {
        "customer_id": 1,
        "age": 25,
        "sex": "Male",
        "location": "London",
        "language": "English",
        "review": "The service was excellent, and the food was delicious.",
        "rating": 5,
        "cuisine_type": "International",
        "restaurant_name": "The Savoy Grill",
        "restaurant_chain": "Gordon Ramsay Restaurants",
        "dish_names": "Beef Wellington, Chocolate Soufflé",
        "city_cuisine_tags": "London Fine Dining, Michelin-starred",
        "dietary_preference": "None"
    },
    {
        "customer_id": 2,
        "age": 30,
        "sex": "Female",
        "location": "Madrid",
        "language": "Spanish",
        "review": "La comida estaba increíble, pero el servicio fue muy lento.",
        "rating": 3,
        "cuisine_type": "Spanish",
        "restaurant_name": "Mercado de San Miguel",
        "restaurant_chain": "Independent",
        "dish_names": "Tapas, Paella, Jamón Ibérico",
        "city_cuisine_tags": "Madrid Traditional Cuisine, Tapas Bar",
        "dietary_preference": "Pescatarian"
    },
    {
        "customer_id": 3,
        "age": 22,
        "sex": "Non-binary",
        "location": "Berlin",
        "language": "German",
        "review": "Das Essen war in Ordnung, aber der Service war nicht gut.",
        "rating": 2,
        "cuisine_type": "German Fusion",
        "restaurant_name": "Markthalle Neun",
        "restaurant_chain": "Street Food Thursday",
        "dish_names": "Currywurst, Schnitzel",
        "city_cuisine_tags": "Berlin Street Food, Urban Dining",
        "dietary_preference": "Vegetarian"
    },
    {
        "customer_id": 4,
        "age": 45,
        "sex": "Male",
        "location": "Paris",
        "language": "French",
        "review": "Le repas était incroyable, et le personnel était très sympathique.",
        "rating": 5,
        "cuisine_type": "French Haute Cuisine",
        "restaurant_name": "Le Jules Verne",
        "restaurant_chain": "Alain Ducasse",
        "dish_names": "Foie Gras, Crème Brûlée",
        "city_cuisine_tags": "Paris Gourmet, Eiffel Tower Dining",
        "dietary_preference": "None"
    },
    {
        "customer_id": 5,
        "age": 35,
        "sex": "Female",
        "location": "Tokyo",
        "language": "Japanese",
        "review": "料理は美味しかったが、サービスが少し遅かったです。",
        "rating": 3,
        "cuisine_type": "Japanese Kaiseki",
        "restaurant_name": "Narisawa",
        "restaurant_chain": "Independent",
        "dish_names": "Satoyama Cuisine, Seasonal Sushi",
        "city_cuisine_tags": "Tokyo Fine Dining, Innovative Cuisine",
        "dietary_preference": "Pescatarian"
    },
    {
        "customer_id": 6,
        "age": 29,
        "sex": "Male",
        "location": "São Paulo",
        "language": "Portuguese",
        "review": "O atendimento foi péssimo, e a comida estava fria.",
        "rating": 1,
        "cuisine_type": "Brazilian",
        "restaurant_name": "Mocotó",
        "restaurant_chain": "Independent",
        "dish_names": "Feijoada, Pão de Queijo",
        "city_cuisine_tags": "São Paulo Street Food, Brazilian Traditional",
        "dietary_preference": "None"
    },
    {
        "customer_id": 7,
        "age": 38,
        "sex": "Female",
        "location": "New York",
        "language": "English",
        "review": "Amazing experience, great food and excellent service.",
        "rating": 5,
        "cuisine_type": "American Contemporary",
        "restaurant_name": "Eleven Madison Park",
        "restaurant_chain": "Make It Nice",
        "dish_names": "Honey Lavender Duck, Milk Chocolate Cremeux",
        "city_cuisine_tags": "New York Fine Dining, Michelin-starred",
        "dietary_preference": "Vegan"
    },
    {
        "customer_id": 8,
        "age": 26,
        "sex": "Male",
        "location": "Beijing",
        "language": "Chinese",
        "review": "食物很好吃，但是服务员态度不太好。",
        "rating": 2,
        "cuisine_type": "Beijing Cuisine",
        "restaurant_name": "Da Dong Roast Duck",
        "restaurant_chain": "Da Dong Group",
        "dish_names": "Peking Duck, Dim Sum",
        "city_cuisine_tags": "Beijing Traditional Cuisine, Gourmet",
        "dietary_preference": "None"
    },
    {
        "customer_id": 9,
        "age": 33,
        "sex": "Female",
        "location": "Moscow",
        "language": "Russian",
        "review": "Еда была вкусной, но обслуживание оставляло желать лучшего.",
        "rating": 3,
        "cuisine_type": "Russian Contemporary",
        "restaurant_name": "White Rabbit",
        "restaurant_chain": "Independent",
        "dish_names": "Beef Stroganoff, Olivier Salad",
        "city_cuisine_tags": "Moscow Fine Dining, Modern Russian",
        "dietary_preference": "Vegetarian"
    },
    {
        "customer_id": 10,
        "age": 40,
        "sex": "Male",
        "location": "Dubai",
        "language": "Arabic",
        "review": "الطعام كان ممتازا والخدمة لا يعلى عليها.",
        "rating": 5,
        "cuisine_type": "Middle Eastern Fusion",
        "restaurant_name": "Pierchic",
        "restaurant_chain": "Al Mahara",
        "dish_names": "Seafood Mandi, Arabic Mezze",
        "city_cuisine_tags": "Dubai Luxury Dining, Waterfront Restaurant",
        "dietary_preference": "None"
    }
]

# Define CSV file name
csv_file = "restaurant_biodata_reviews_with_entities.csv"

# Write data to CSV
with open(csv_file, mode="w", newline="", encoding="utf-8") as file:
    fieldnames = ["customer_id", "age", "sex", "location", "language", "review", "rating", 
                  "cuisine_type", "restaurant_name", "restaurant_chain", "dish_names", 
                  "city_cuisine_tags", "dietary_preference"]
    writer = csv.DictWriter(file, fieldnames=fieldnames)

    writer.writeheader()  # Write header row
    writer.writerows(data)  # Write data rows

print(f"CSV file '{csv_file}' with restaurant biodata, reviews, and entity linking columns has been created.")

CSV file 'restaurant_biodata_reviews_with_entities.csv' with restaurant biodata, reviews, and entity linking columns has been created.


In [ ]:
# Get the current working directory
cwd = os.getcwd()
print("Current working directory:", cwd)

# Path to your CSV file
csv_file_path = "restaurant_biodata_reviews.csv"   # Define the path to the CSV file


# Read CSV data using DataManager
try:
    df = pd.read_csv(csv_file_path)  # Read the CSV file into a DataFrame using DataManager
    print("CSV file imported successfully!")  # Print success message
    display(df.head())  # Display the first few rows of the DataFrame
except Exception as e:
    print(f"Error importing CSV file: {e}")  # Print error message if CSV import fails